In [ ]:
# Minimal GRU for univariate next-step prediction (Colab-ready)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# --- load data (assumes common_dataset.csv with column 'value') ---
df = pd.read_csv('dataset.csv')
series = df['value'].astype('float32').values.reshape(-1, 1)

# --- scale to [0,1] ---
scaler = MinMaxScaler()
series_s = scaler.fit_transform(series)

# --- create sequences (seq_len timesteps -> next value) ---
seq_len = 10
X, y = [], []
for i in range(len(series_s) - seq_len):
    X.append(series_s[i:i + seq_len])
    y.append(series_s[i + seq_len])
X = np.array(X)   # shape: (samples, seq_len, 1)
y = np.array(y)   # shape: (samples, 1)

# --- train/test split (80/20) ---
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# --- build GRU model ---
model = Sequential([
    GRU(32, input_shape=(seq_len, 1)),  # 32 GRU units
    Dense(1)                            # predict single next value
])
model.compile(optimizer='adam', loss='mse')

# --- train (minimal epochs) ---
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# --- predict and invert scaling ---
pred_s = model.predict(X_test)
pred = scaler.inverse_transform(pred_s)
y_true = scaler.inverse_transform(y_test)

# --- show sample predictions ---
for i in range(min(10, len(pred))):
    print(f"Predicted: {pred[i,0]:.4f} | Actual: {y_true[i,0]:.4f}")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 137ms/step - loss: 0.3050 - val_loss: 0.8252
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.2223 - val_loss: 0.5733
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.1274 - val_loss: 0.3706
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0719 - val_loss: 0.2126
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0376 - val_loss: 0.1000
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0214 - val_loss: 0.0358
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0204 - val_loss: 0.0154
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0245 - val_loss: 0.0156
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0173 - val_loss: 0.0235
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0181 - val_loss: 0.0366
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
Predicted: 118.5783 | Actual: 124.2626
Predicted: 118.9923 | Actual: 140.6077
Predicted: 117.7552 | Actual: 134.2758
Predicted: 1